In [1]:
import pandas as pd

In [2]:
df= pd.read_csv("/content/Train_Split.csv")
df.drop("variance",axis=1,inplace=True)

In [3]:
df.head()

,skewness,curtosis,entropy,y
0,7.2590,0.070827,-7.30040,1
1,3.3998,-4.432700,-4.26550,1
2,-9.2828,7.719000,-1.71680,1
3,-3.6792,3.828100,1.62970,0
4,6.8369,0.697180,-0.55691,0


In [4]:
df_x = df.drop('y',axis=1)
df_y = df['y']

In [5]:
from sklearn.metrics import log_loss,make_scorer


In [38]:
from tpot import TPOTClassifier
pipeline_optimizer = TPOTClassifier(generations=20, population_size=100, cv=5,
                                    random_state=42, verbosity=2)


In [39]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.2, random_state=42)


In [40]:
pipeline_optimizer.fit(X_train, y_train)


Optimization Progress:   0%|          | 0/2100 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.9726298701298701

Generation 2 - Current best internal CV score: 0.9726298701298701

Generation 3 - Current best internal CV score: 0.9726298701298701

Generation 4 - Current best internal CV score: 0.9726298701298701

Generation 5 - Current best internal CV score: 0.9726298701298701

Generation 6 - Current best internal CV score: 0.9726298701298701

Generation 7 - Current best internal CV score: 0.9726298701298701

Generation 8 - Current best internal CV score: 0.9737597402597402

Generation 9 - Current best internal CV score: 0.9737597402597402

Generation 10 - Current best internal CV score: 0.9737597402597402

Generation 11 - Current best internal CV score: 0.9737597402597402

Generation 12 - Current best internal CV score: 0.9737597402597402

Generation 13 - Current best internal CV score: 0.9737597402597402

Generation 14 - Current best internal CV score: 0.9737597402597402

Generation 15 - Current best internal CV score: 0.973772

TPOTClassifier(generations=20, random_state=42, verbosity=2)

In [41]:
predictions = pipeline_optimizer.predict_proba(X_test)


In [42]:
predictions


array([[1.        , 0.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.4367988 , 0.5632012 ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.68036415, 0.31963585],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.32393627, 0.67606373],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.18307642, 0.81692358],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.70666941, 0.29333059],
       [0.

In [43]:
pipeline_optimizer.fitted_pipeline_

Pipeline(steps=[('kneighborsclassifier',
                 KNeighborsClassifier(p=1, weights='distance'))])

In [44]:
import pickle
filename = 'tpot_model'
pickle.dump(pipeline_optimizer.fitted_pipeline_, open(filename, 'wb'))

In [45]:
with open(filename, 'rb') as file:
    loaded_model = pickle.load(file)

In [46]:
predictions_2=loaded_model.predict(X_test)

In [47]:
report_1 = classification_report(y_test,predictions_2)
print('Classification Report:\n', report_1)


Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.94      0.95       122
           1       0.93      0.96      0.94        98

    accuracy                           0.95       220
   macro avg       0.95      0.95      0.95       220
weighted avg       0.95      0.95      0.95       220



In [52]:
from sklearn.metrics import f1_score
f1 = f1_score(y_test, predictions_2)

print(f'F1: {f1}')


F1: 0.9447236180904524


In [49]:
from sklearn.calibration import CalibratedClassifierCV
calibrated_model = CalibratedClassifierCV(estimator=loaded_model, method='sigmoid', cv='prefit')
calibrated_model.fit(X_train, y_train)
calibrated_predictions = calibrated_model.predict(X_test)


In [50]:
from sklearn.metrics import accuracy_score, classification_report

calibrated_f1 = f1_score(y_test, calibrated_predictions)
print(f'F1: {calibrated_f1}')

report = classification_report(y_test, calibrated_predictions)
print('Classification Report:\n', report)


F1: 0.9447236180904524
Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.94      0.95       122
           1       0.93      0.96      0.94        98

    accuracy                           0.95       220
   macro avg       0.95      0.95      0.95       220
weighted avg       0.95      0.95      0.95       220



In [51]:
calibrated_model.predict_proba(X_test)

array([[0.99795501, 0.00204499],
       [0.00255102, 0.99744898],
       [0.99795501, 0.00204499],
       [0.99795501, 0.00204499],
       [0.99795501, 0.00204499],
       [0.00255102, 0.99744898],
       [0.99795501, 0.00204499],
       [0.00255102, 0.99744898],
       [0.34126528, 0.65873472],
       [0.99795501, 0.00204499],
       [0.99795501, 0.00204499],
       [0.00255102, 0.99744898],
       [0.99795501, 0.00204499],
       [0.99795501, 0.00204499],
       [0.00255102, 0.99744898],
       [0.90919584, 0.09080416],
       [0.00255102, 0.99744898],
       [0.00255102, 0.99744898],
       [0.99795501, 0.00204499],
       [0.11609317, 0.88390683],
       [0.99795501, 0.00204499],
       [0.00255102, 0.99744898],
       [0.99795501, 0.00204499],
       [0.00255102, 0.99744898],
       [0.99795501, 0.00204499],
       [0.00255102, 0.99744898],
       [0.02314236, 0.97685764],
       [0.99795501, 0.00204499],
       [0.00255102, 0.99744898],
       [0.93237149, 0.06762851],
       [0.

In [53]:
import pickle
filename = 'calibrated_tpot_model'
pickle.dump(calibrated_model, open(filename, 'wb'))